<h1>New Experiment 2: Random rejection of motion-free subjects</h1>
In this notebook, random rejection tests were conducted using data from S1 in the still condition. The generalisation tests were conducted over S2. Each time we fix the number of retained volumes N. We randomly select N volumes from a subject's data, then we train our proposed model over this sample. At testing, we applied the same rejection scheme to S2, then test over the subset of S2.</br>
We would like to show this proposed model is robust, by which it is independent of 1. The proportion of discarded volumes and 2. The different combinations of b-values.</br>
<h2>Packages</h2>

In [1]:
import nibabel as nib
import numpy as np
"""
packages that does conventional model fitting
"""
import amico
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser
"""
packages that trains network
"""
from Training import train_network
from utils.model import parser as model_parser
"""
packages that test network
"""
from Testing import test_model
"""
packages that produce the rejection shceme
"""
from filter_qa import parser as filter_parser, load_eddy
"""
package to store the intermediate result
"""
import pickle

2022-06-21 09:45:19.335025: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
"""
packages that handle graphs
"""
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from skimage.metrics import structural_similarity as compare_ssim
from utils import calc_ssim
%matplotlib inline
def plot_loss(cmd):
    """
    A function that used to plot the loss curve for the trained network.
    Args:
        cmd: String, the command line in the terminal
    """
    args = model_parser().parse_args(cmd.split())
    history = train_network(args)
    # plt.plot(history.history['loss'])
    # plt.plot(history.history['val_loss'])
    # plt.title('model loss')
    # plt.ylabel('loss')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'validation'], loc='upper left')
    # plt.show()

def show_slices(slices, grayscale=True):
    """
    Function to display the slices

    Args:
        slices (list): a list of 2d ndarray that contains the data to be displayed
        grayscale (bool, optional): True, if diplay grayscale img. Defaults to True.
    """    
    fig, axes = plt.subplots(1, len(slices), figsize=(10,10))
    cax = fig.add_axes([0, 0, .3, .3])
    for i, slice in enumerate(slices):
        # use grayscale for displaying ref and pred imgs:
        if grayscale:
            cmap = plt.get_cmap('gray')
            cmaplist = [cmap(i) for i in range(cmap.N)]
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            bounds = np.arange(0, 1.0, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        else:
            # define the colormap
            cmap = plt.get_cmap('bwr')
            # extract all colors from the .jet map
            cmaplist = [cmap(i) for i in range(cmap.N)]
            # create the new map
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            # define the bins and normalize and forcing 0 to be part of the colorbar
            # define the min and max to be -1 and +1 respectively
            bounds = np.arange(-0.5, 0.5, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        fig.colorbar(im, cax=cax, orientation='vertical')

def scale(img):
    # for i in range(img.shape[0]):
    #     for j in range(img.shape[1]):
    #         img[i][j] = ((img[i][j]+1)/2)*255
    return img

def compare_simi(pred, ref):
    return calc_ssim(pred, ref)

In [3]:
def visualise(ref_ndi, ref_odi, ref_fwf, retained_vol, subject, model, layer, affine1, affine2, affine3):
    """
    Function to visualise the imgs and difference maps

    Args:
        ref_ndi (ndarray): the reference NDI data
        ref_odi (ndarray): the reference ODI data
        ref_fwf (ndarray): the reference FWF data
        retained_vol (int): the number of volumes retained after rejection
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refNDI0 = ref_ndi[26, :, :]
    refNDI1 = ref_ndi[:, 30, :]
    refNDI2 = ref_ndi[:, :, 16]
    show_slices([refNDI0, refNDI1, refNDI2])
    plt.suptitle("Center slices for NDI reference image")

    refODI0 = ref_odi[26, :, :]
    refODI1 = ref_odi[:, 30, :]
    refODI2 = ref_odi[:, :, 16]
    show_slices([refODI0, refODI1, refODI2])
    plt.suptitle("Center slices for ODI reference image")

    refFWF0 = ref_fwf[26, :, :]
    refFWF1 = ref_fwf[:, 30, :]
    refFWF2 = ref_fwf[:, :, 16]
    show_slices([refFWF0, refFWF1, refFWF2])
    plt.suptitle("Center slices for FWF reference image")

    # visualise the pred imgs produced at varied input size
    ndi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI.nii'
    odi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI.nii'
    fwf_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF.nii'
    ndi_img = nib.load(ndi_path)
    ndi_data = ndi_img.get_fdata()
    odi_img = nib.load(odi_path)
    odi_data = odi_img.get_fdata()
    fwf_img = nib.load(fwf_path)
    fwf_data = fwf_img.get_fdata()

    ndi0 = ndi_data[26, :, :]
    ndi1 = ndi_data[:, 30, :]
    ndi2 = ndi_data[:, :, 16]
    show_slices([ndi0, ndi1, ndi2])
    plt.suptitle('Center slices for NDI predicted image by '+model+', input size='+str(retained_vol))
    (score, ndidiff) = compare_ssim(ndi_data, ref_ndi, full=True)
    print(str(retained_vol)+'input size the ssim score for ndi is: ' + str(score))

    odi0 = odi_data[26, :, :]
    odi1 = odi_data[:, 30, :]
    odi2 = odi_data[:, :, 16]
    show_slices([odi0, odi1, odi2])
    plt.suptitle('Center slices for ODI predicted image by '+model+', input size='+str(retained_vol))
    (score, odidiff) = compare_ssim(odi_data, ref_odi, full=True)
    print(str(retained_vol)+'input size the ssim score for odi is: ' + str(score))

    fwf0 = fwf_data[26, :, :]
    fwf1 = fwf_data[:, 30, :]
    fwf2 = fwf_data[:, :, 16]
    show_slices([fwf0, fwf1, fwf2])
    plt.suptitle('Center slices for FWF predicted image by '+model+', input size='+str(retained_vol))
    (score, fwfdiff) = compare_ssim(fwf_data, ref_fwf, full=True)
    print(str(retained_vol)+'input size the ssim score for fwf is: ' + str(score))

    # plot the difference map between the imgs by the lib
    ndidiff0 = scale(refNDI0 - ndi0)
    ndidiff1 = scale(refNDI1 - ndi1)
    ndidiff2 = scale(refNDI2 - ndi2)
    show_slices([ndidiff0, ndidiff1, ndidiff2], grayscale=False)
    plt.suptitle("Difference map NDI")

    diff_img_np = ref_ndi - ndi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine1)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI_difference.nii')

    odidiff0 = scale(refODI0 - odi0)
    odidiff1 = scale(refODI1 - odi1)
    odidiff2 = scale(refODI2 - odi2)
    show_slices([odidiff0, odidiff1, odidiff2], grayscale=False)
    plt.suptitle("Difference map ODI")

    diff_img_np = ref_odi - odi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine2)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI_difference.nii')

    fwfdiff0 = scale(refFWF0 - fwf0)
    fwfdiff1 = scale(refFWF1 - fwf1)
    fwfdiff2 = scale(refFWF2 - fwf2)
    show_slices([fwfdiff0, fwfdiff1, fwfdiff2], grayscale=False)
    plt.suptitle("Difference map FWF")

    diff_img_np = ref_fwf - fwf_data
    diff_img = nib.Nifti1Image(diff_img_np, affine3)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF_difference.nii')

<h2>The class for parsing command line arguments</h2>

In [4]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

<h2>Data Preprocessing</h2>

In [5]:
# motion-free subject path
s01_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/'
s02_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/'
# motion-free target labels
s01_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_NDI.nii'
s02_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_NDI.nii'

s01_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_ODI.nii'
s02_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_ODI.nii'

s01_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_FWF.nii'
s02_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_FWF.nii'

In [6]:
def filter_mask(subpath, fwfpath, threshold=0.99):
    """
    By looking at the imgs generated, we have found out there are some regions that should not be included. Since they have values higher than 1.0
    And we have found out voxels have NDI and ODI values, while that voxel has GROUND TRUTH FWF 1.0
    This should indicate that that voxel should not even be included in the training
    Therefore we want to filter the each subject's mask first, by using their corresponding GROUND TRUTH FWF

    Args:
        subpath (string): the path of the subject folder
        fwfpath (string): the path of the corresponding fwf file
        threshold (float): the thresholds to be used to filter of the mask,
                           a stringnent threshold would be 0.9, the least stringnent threshold is 1.0
                           by default, it is set to 0.99
    """
    # fetch the mask data
    img_mask = nib.load(subpath+'mask-e.nii')
    original_mask = img_mask.get_fdata()
    original_affine = img_mask.affine
    shape = original_mask.shape # retain the shape of the mask
    origin_nonzeros = np.count_nonzero(original_mask)
    print('original mask has: ' + str(origin_nonzeros) + ' of nonzero voxels')
    # fetch the FWF data
    fwf = nib.load(fwfpath).get_fdata()
    # filter
    mask = original_mask.flatten() # this makes a copy of the orginal mask
    fwf = fwf.reshape(mask.shape[0]) # reshape fwf to the corresponding shape
    for i in range(len(mask)):
        # if fwf has high value, means there is no tissue
        # therefore, the voxel should be excluded
        if fwf[i] >= threshold:
            mask[i] = 0.0
    # reshape mask back
    mask = mask.reshape(shape)
    filter_nonzeros = np.count_nonzero(mask)
    print('filtered mask has: ' +str(filter_nonzeros) + ' of nonzero voxels')
    # save the mask
    filter_img = nib.Nifti1Image(mask, original_affine)
    nib.save(filter_img, subpath+'filtered_mask.nii')

In [7]:
# Use the above code to filter each subject's mask. Store as filtered_mask.nii in each subject folder
filter_mask(s01_path, s01_FWF_path)
filter_mask(s02_path, s02_FWF_path)

original mask has: 82889 of nonzero voxels
filtered mask has: 81882 of nonzero voxels
original mask has: 94198 of nonzero voxels
filtered mask has: 91401 of nonzero voxels


In [8]:
# filtered mask path for each subject
s01_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/filtered_mask.nii'
s02_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/filtered_mask.nii'

In [9]:
"""
Generate the base dataset for s01_still and s02_still for all NODDI parameters.
"""
cmd = "--base --label_type A --subjects s01_still s02_still"
args = data_parser().parse_args(cmd.split())
generate_data(args)

Generating for s01_still ...
base data dataset has shape: (84, 84, 50, 96)
base label dataset has shape: (84, 84, 50, 3)
Generating for s02_still ...
base data dataset has shape: (84, 84, 50, 96)
base label dataset has shape: (84, 84, 50, 3)


In [10]:
ltype = 'A' # the NODDI parameter we want to estimate, where A stands for ALL
model = 'conv3d'
# firstly, the 3d cnn dataset is required to be generated.
# the cmd line code for generating 3d cnn data
cnn3ddata_cmd = '--subjects s01_still s02_still' + ' --label_type ' + ltype + ' --' + model
cnn3ddata_args  = data_parser().parse_args(cnn3ddata_cmd.split())
generate_data(cnn3ddata_args)

Generating for s01_still ...
mask has shape: (82, 82, 48)
data has shape: (84, 84, 50, 96)
label has shape: (82, 82, 48, 3)
(84, 84, 50, 96) (82, 82, 48)
(81634, 3, 3, 3, 96)
saved patches has shape: (81634, 2592)
(82, 82, 48, 3) (82, 82, 48)
(81634, 1, 1, 1, 3)
svaed labels has shape: (81634, 1, 1, 1, 3)
Generating for s02_still ...
mask has shape: (82, 82, 48)
data has shape: (84, 84, 50, 96)
label has shape: (82, 82, 48, 3)
(84, 84, 50, 96) (82, 82, 48)
(90846, 3, 3, 3, 96)
saved patches has shape: (90846, 2592)
(82, 82, 48, 3) (82, 82, 48)
(90846, 1, 1, 1, 3)
svaed labels has shape: (90846, 1, 1, 1, 3)


In [11]:
"""
Using nib to fetch the ground truth img
"""
#load the truth data for subject 1
s01_NDI_img = nib.load(s01_NDI_path)
s01_ODI_img = nib.load(s01_ODI_path)
s01_FWF_img = nib.load(s01_FWF_path)
s01_NDI_affine = s01_NDI_img.affine
s01_ODI_affine = s01_ODI_img.affine
s01_FWF_affine = s01_FWF_img.affine
s01_NDI_img_data = s01_NDI_img.get_fdata()
s01_ODI_img_data = s01_ODI_img.get_fdata()
s01_FWF_img_data = s01_FWF_img.get_fdata()
#load the truth data for subject 2
s02_NDI_img = nib.load(s02_NDI_path)
s02_ODI_img = nib.load(s02_ODI_path)
s02_FWF_img = nib.load(s02_FWF_path)
s02_NDI_affine = s02_NDI_img.affine
s02_ODI_affine = s02_ODI_img.affine
s02_FWF_affine = s02_FWF_img.affine
s02_NDI_img_data = s02_NDI_img.get_fdata()
s02_ODI_img_data = s02_ODI_img.get_fdata()
s02_FWF_img_data = s02_FWF_img.get_fdata()

------------------------
<h2>Random Rejection Scheme Setup</h2>
Let N to be the tested number of retained volumes. For N, 100 subsampled schemes were drawn randomly from the full scheme (with the first b=0 volume and at least two different b values always included). Each subsampled schem was used to evaluate both techniques. For both 3D CNN and AMICO, N are 60, 40 and 30. We would give further undersampled scheme to 3D CNN, N are 20, 16 and 12.

In [14]:
# a list contains the N for each tested retained volume
N = [60, 40, 30, 20, 16, 12]
random_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}
random_dict

{60: [], 40: [], 30: [], 20: [], 16: [], 12: []}

In [ ]:
# Generated the random scheme for each tested volume
for n in N:
    # for each retained volume, we create 100 subsamples
    for i in range(100):
        ones = np.ones(n-1) # 1 = retained
        zeros = np.zeros(96-n)

        scheme = np.random.choice(np.concatenate([ones,zeros]), 95, replace=False)
        # always select the first b=0 volume
        scheme = np.insert(scheme, 0, 1.0)
        scheme = ' '.join(map(str, scheme))
        random_dict[n].append(scheme)

In [ ]:
# save the rejection scheme
random_60_file = 'random_60.pickle'
random_40_file = 'random_40.pickle'
random_30_file = 'random_30.pickle'
random_20_file = 'random_20.pickle'
random_16_file = 'random_16.pickle'
random_12_file = 'random_12.pickle'

with open(random_60_file, 'wb') as handle:
    pickle.dump(random_dict[60], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_40_file, 'wb') as handle:
    pickle.dump(random_dict[40], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_30_file, 'wb') as handle:
    pickle.dump(random_dict[30], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_20_file, 'wb') as handle:
    pickle.dump(random_dict[20], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_16_file, 'wb') as handle:
    pickle.dump(random_dict[16], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(random_12_file, 'wb') as handle:
    pickle.dump(random_dict[12], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
cnn_rmse_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}
cnn_ssim_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}

----------------------------
<h2>Training</h2>
Hyperparameters

In [13]:
# hyperparameters for the networks
layers = 4 # the number of hidden layers; this is the optimal number of layer
lr = 0.0001 # the learning rate
patch_size = 3 # the size of patches for 2D and 3D CNN
batch = 256 # the batch size
epoch = 100 # the number of epoches for training

# we firstly study subject s02, use s01 as a seperate training dataset
study_subject = 's02_still' # the subject we want to study
sep_train_subject = 's01_still' # this is separate training candidate

<h4>N = 60</h4>

In [ ]:
for i in range(len(random_dict[60])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[60][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[60].append(RMSE)
    cnn_ssim_dict[60].append(SSIM)

In [ ]:
with open('cnn_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[60])//2, len(random_dict[60])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[60][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[60].append(RMSE)
    cnn_ssim_dict[60].append(SSIM)

In [ ]:
with open('cnn_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 40</h4>

In [ ]:
for i in range(len(random_dict[40])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[40][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[40].append(RMSE)
    cnn_ssim_dict[40].append(SSIM)

In [ ]:
with open('cnn_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[40])//2, len(random_dict[40])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[40][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[40].append(RMSE)
    cnn_ssim_dict[40].append(SSIM)

In [ ]:
with open('cnn_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 30</h4>

In [ ]:
for i in range(len(random_dict[30])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[30][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[30].append(RMSE)
    cnn_ssim_dict[30].append(SSIM)

In [ ]:
with open('cnn_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[30])//2, len(random_dict[30])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[30][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[30].append(RMSE)
    cnn_ssim_dict[30].append(SSIM)

In [ ]:
with open('cnn_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 20</h4>

In [ ]:
for i in range(len(random_dict[20])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[20][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[20].append(RMSE)
    cnn_ssim_dict[20].append(SSIM)

In [ ]:
with open('cnn_rmse_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_20_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[20])//2, len(random_dict[20])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[20][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[20].append(RMSE)
    cnn_ssim_dict[20].append(SSIM)

In [ ]:
with open('cnn_rmse_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 16</h4>

In [ ]:
for i in range(len(random_dict[16])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[16][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[16].append(RMSE)
    cnn_ssim_dict[16].append(SSIM)

In [ ]:
with open('cnn_rmse_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_16_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[16])//2, len(random_dict[16])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[16][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[16].append(RMSE)
    cnn_ssim_dict[16].append(SSIM)

In [ ]:
with open('cnn_rmse_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 12</h4>

In [26]:
# read the rejection scheme
random_12 = []
with (open('/home/vw/Desktop/IndividualProject/MedICSS2021_/Net/random_12.pickle', "rb")) as openfile:
    while True:
        try:
            random_12.append(pickle.load(openfile))
        except EOFError:
            break
random_12 = np.array(random_12[0])

In [28]:
# for i in range(len(random_dict[12])//2):
for i in range(len(random_12)//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    # movefile = random_dict[12][i]
    movefile = random_12[i]
    # movefile = movefile.split()
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[12].append(RMSE)
    cnn_ssim_dict[12].append(SSIM)

12
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3, 3, 3, 12)]     0         
_________________________________________________________________
conv3d (Conv3D)              (None, 1, 1, 1, 150)      48750     
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 1, 1, 1, 3)    

2022-06-21 09:52:21.625809: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-06-21 09:52:21.653029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-21 09:52:21.653121: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.777GHz coreCount: 28 deviceMemorySize: 11.76GiB deviceMemoryBandwidth: 335.32GiB/s
2022-06-21 09:52:21.653153: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-06-21 09:52:21.669888: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-06-21 09:52:21.692325: I tensorflow/stream_executor/pl

The remained data has shape: (81634, 3, 3, 3, 12)
(81634, 3, 3, 3, 12)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 150)
(81634, 1, 1, 1, 3)
(81634, 1, 1, 1, 3)
Training start ...


2022-06-21 09:52:25.061988: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 09:52:25.062739: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1391] Profiler found 1 GPUs
2022-06-21 09:52:25.067134: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcupti.so.10.1
2022-06-21 09:52:25.070342: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


...

2022-06-21 10:39:30.155329: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 10:39:30.177831: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-06-21 10:39:30.178139: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-06-21 10:39:30.180461: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_06_21_10_39_30
2022-06-21 10:39:30.181105: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_06_21_10_39_30/vw.trace.json.gz
2022-06-21 10:39:30.181185: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-06-21 10:39:30.183262: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

12-conv3d-patch_3-base_1-layer_4-label_NDIODIFWFsynthetic
Model: "functional_293"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_98 (InputLayer)        [(None, 84, 84, 50, 12)]  0         
_________________________________________________________________
conv3d_485 (Conv3D)          (None, 82, 82, 48, 150)   48750     
_________________________________________________________________
conv3d_486 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_487 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_488 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_97 (Dropout)         (None, 82, 82, 48, 150)   0         
____________________________________________________________

2022-06-21 10:40:31.638305: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 10:40:31.638422: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-06-21 10:40:32.225525: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 10:40:32.266448: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-06-21 10:40:32.266917: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-06-21 10:40:32.269997: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_06_21_10_40_32
2022-06-21 10:40:32.270802: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_06_21_10_40_32/vw.trace.json.gz
2022-06-21 10:40:32.271085: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-06-21 10:40:32.273772: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

12-conv3d-patch_3-base_1-layer_4-label_NDIODIFWFsynthetic
Model: "functional_299"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_100 (InputLayer)       [(None, 84, 84, 50, 12)]  0         
_________________________________________________________________
conv3d_495 (Conv3D)          (None, 82, 82, 48, 150)   48750     
_________________________________________________________________
conv3d_496 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_497 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_498 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_99 (Dropout)         (None, 82, 82, 48, 150)   0         
____________________________________________________________

In [29]:
with open('cnn_rmse_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_12_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
# for i in range(len(random_dict[12])//2, len(random_dict[12])):
for i in range(len(random_12)//2, len(random_12)):
    # we have 100 movefiles for each retained tested volume
    # Training
    # movefile = random_dict[12][i]
    movefile = random_12[i]
    # movefile = movefile.split()
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[12].append(RMSE)
    cnn_ssim_dict[12].append(SSIM)

12
Model: "functional_301"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_101 (InputLayer)       [(None, 3, 3, 3, 12)]     0         
_________________________________________________________________
conv3d_500 (Conv3D)          (None, 1, 1, 1, 150)      48750     
_________________________________________________________________
conv3d_501 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_502 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
conv3d_503 (Conv3D)          (None, 1, 1, 1, 150)      22650     
_________________________________________________________________
dropout_100 (Dropout)        (None, 1, 1, 1, 150)      0         
_________________________________________________________________
conv3d_504 (Conv3D)          (None, 1, 1, 1, 3)  

2022-06-21 10:41:33.694435: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 10:41:33.694516: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-06-21 10:41:34.207231: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 10:41:34.241022: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-06-21 10:41:34.241446: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-06-21 10:41:34.244164: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_06_21_10_41_34
2022-06-21 10:41:34.244903: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_06_21_10_41_34/vw.trace.json.gz
2022-06-21 10:41:34.245103: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-06-21 10:41:34.247417: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

12-conv3d-patch_3-base_1-layer_4-label_NDIODIFWFsynthetic
Model: "functional_305"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_102 (InputLayer)       [(None, 84, 84, 50, 12)]  0         
_________________________________________________________________
conv3d_505 (Conv3D)          (None, 82, 82, 48, 150)   48750     
_________________________________________________________________
conv3d_506 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_507 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_508 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_101 (Dropout)        (None, 82, 82, 48, 150)   0         
____________________________________________________________

2022-06-21 10:42:32.377527: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 10:42:32.377604: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


...

2022-06-21 11:28:48.151688: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 11:28:48.187186: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-06-21 11:28:48.187614: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-06-21 11:28:48.190343: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_06_21_11_28_48
2022-06-21 11:28:48.191042: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_06_21_11_28_48/vw.trace.json.gz
2022-06-21 11:28:48.191225: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-06-21 11:28:48.193558: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

12-conv3d-patch_3-base_1-layer_4-label_NDIODIFWFsynthetic
Model: "functional_593"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_198 (InputLayer)       [(None, 84, 84, 50, 12)]  0         
_________________________________________________________________
conv3d_985 (Conv3D)          (None, 82, 82, 48, 150)   48750     
_________________________________________________________________
conv3d_986 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_987 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_988 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_197 (Dropout)        (None, 82, 82, 48, 150)   0         
____________________________________________________________

2022-06-21 11:29:50.647496: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 11:29:50.647588: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed


2022-06-21 11:29:51.342218: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-06-21 11:29:51.372916: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1513] CUPTI activity buffer flushed
2022-06-21 11:29:51.373283: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-06-21 11:29:51.375776: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022_06_21_11_29_51
2022-06-21 11:29:51.376430: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_06_21_11_29_51/vw.trace.json.gz
2022-06-21 11:29:51.376537: E tensorflow/core/profiler/utils/hardware_type_utils.cc:61] Invalid GPU compute capability.
2022-06-21 11:29:51.378855: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: logs/train/plugins/profile/2022

12-conv3d-patch_3-base_1-layer_4-label_NDIODIFWFsynthetic
Model: "functional_599"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_200 (InputLayer)       [(None, 84, 84, 50, 12)]  0         
_________________________________________________________________
conv3d_995 (Conv3D)          (None, 82, 82, 48, 150)   48750     
_________________________________________________________________
conv3d_996 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_997 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
conv3d_998 (Conv3D)          (None, 82, 82, 48, 150)   22650     
_________________________________________________________________
dropout_199 (Dropout)        (None, 82, 82, 48, 150)   0         
____________________________________________________________

In [31]:
with open('cnn_rmse_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)